In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import datetime
import arrow
import os
import pandas as pd
import warnings
import time

from util.file_util import StockTwitsFileReader
from util.stocktwits_helper import (
    get_est_market_datetime, get_daily_sentiment_count_df,
    get_all_stocktwits_tickers)

In [4]:
from arrow.factory import ArrowParseWarning

In [5]:
# Disable Arrow warnings per https://github.com/crsmithdev/arrow/issues/612
warnings.simplefilter("ignore", ArrowParseWarning)

In [6]:
stock_twits_reader = StockTwitsFileReader()

In [10]:
ticker = 'NVDA'

In [11]:
start_date = '2018-01-01'
end_date = '2018-03-30'

In [12]:
df = stock_twits_reader.read_twit_file_in_range(ticker, start_date, end_date, cols='default')

In [13]:
daily_sentiment_df = get_daily_sentiment_count_df(df)

In [17]:
df[['date_est', 'created_at_est', 'mkt_date']].head(10)

,date_est,created_at_est,mkt_date
0,2018-01-01,2018-01-01 23:55:19,2018-01-02
1,2018-01-01,2018-01-01 23:51:25,2018-01-02
2,2018-01-01,2018-01-01 23:25:56,2018-01-02
3,2018-01-01,2018-01-01 23:19:24,2018-01-02
4,2018-01-01,2018-01-01 22:33:12,2018-01-02
5,2018-01-01,2018-01-01 21:55:32,2018-01-02
6,2018-01-01,2018-01-01 21:44:45,2018-01-02
7,2018-01-01,2018-01-01 20:54:29,2018-01-02
8,2018-01-01,2018-01-01 20:50:46,2018-01-02
9,2018-01-01,2018-01-01 20:45:07,2018-01-02


In [19]:
get_daily_sentiment_count_df??

Signature:
get_daily_sentiment_count_df(
    stocktwit_raw_df,
    mkt_date_col='mkt_date',
    sentiment_col='entities.sentiment.basic',
)
Docstring: <no docstring>
Source:   
def get_daily_sentiment_count_df(stocktwit_raw_df,
                                 mkt_date_col='mkt_date',
                                 sentiment_col='entities.sentiment.basic'):
    append_market_date_col(stocktwit_raw_df, mkt_date_col=mkt_date_col)

    grouped = stocktwit_raw_df.groupby(
        mkt_date_col)[sentiment_col].value_counts().to_frame('count').reset_index()
    daily_sentiment_count_df = grouped.pivot(index=mkt_date_col,
                                             columns=sentiment_col,
                                             values='count').fillna(0)
    daily_sentiment_count_df.columns.name = None
    daily_sentiment_count_df = daily_sentiment_count_df.sort_index()

    return daily_sentiment_count_df
File:      ~/Personal/Research/Stock_Sentiment/project/stock-sentiment/util/stockt

In [15]:
daily_sentiment_df.head(10)

,Bearish,Bullish
mkt_date,,
2018-01-01,1.0,4.0
2018-01-02,11.0,98.0
2018-01-03,31.0,342.0
2018-01-04,111.0,273.0
2018-01-05,36.0,267.0
2018-01-06,3.0,41.0
2018-01-07,0.0,24.0
2018-01-08,30.0,311.0
2018-01-09,28.0,289.0


## Save down sentiment time series

In [20]:
start_date = '2018-01-01'
end_date = '2019-08-10'

#### Run this for all tickers

In [22]:
from config import STOCKTWITS_TICKER_LIST

In [27]:
ticker_list = STOCKTWITS_TICKER_LIST
len(ticker_list)

502

In [28]:
# ticker_list = get_all_stocktwits_tickers()
# ticker_list = [
#     'UBER',  # Uber
#     'LYFT',  # Lyft
#     'WORK',  # Slack
#     'ZM',  # Zoom
#     'PINS',  # Pinterest
#     'CHWY',  # Chewy
#     'CRWD',  # Crowdstrike
# ]

In [29]:
root_dir = stock_twits_reader.get_root_dir()
save_dir = os.path.join(root_dir, 'processed', 'sentiment_summary_prelim')
save_dir

'/Users/seung-jae_bang/Personal/Research/Stock_Sentiment/data/Stocktwits/processed/sentiment_summary_prelim'

In [30]:
save_file_format = '{ticker}_sentiment_summary.pkl'
save_path_format = os.path.join(save_dir, save_file_format)
save_path_format

'/Users/seung-jae_bang/Personal/Research/Stock_Sentiment/data/Stocktwits/processed/sentiment_summary_prelim/{ticker}_sentiment_summary.pkl'

In [31]:
for t in ticker_list:
    tic = time.time()
    
    raw_df = stock_twits_reader.read_twit_file_in_range(t, start_date, end_date, cols='default')
    daily_sentiment_df = get_daily_sentiment_count_df(raw_df)
    daily_sentiment_df.to_pickle(save_path_format.format(ticker=t))
    
    toc = time.time()
    print('Run for {} took {} mins'.format(t, (toc - tic) / 60))

Run for MPC took 0.30187811851501467 mins
Run for KMX took 0.2525624672571818 mins
Run for REG took 0.14173930088678996 mins
Run for VMC took 0.24179233312606813 mins
Run for NRG took 0.24612401723861693 mins
Run for HBI took 0.23499484856923422 mins
Run for TDG took 0.21144403219223024 mins
Run for NSC took 0.23829656442006428 mins
Run for ETN took 0.21045690377553303 mins
Run for MDT took 0.3957903822263082 mins
Run for NEM took 0.3401297330856323 mins
Run for XOM took 0.469477383295695 mins
Run for WY took 0.20032191673914593 mins
Run for COTY took 0.27649683157602944 mins
Run for ARE took 0.1323761026064555 mins
Run for ADS took 0.18617796897888184 mins
Run for UAL took 0.42787279685338336 mins
Run for MMM took 0.370194681485494 mins
Run for NEE took 0.2887084682782491 mins
Run for TXN took 0.37742815017700193 mins
Run for HUM took 0.3250874320665995 mins
Run for IPGP took 0.2541824261347453 mins
Run for FFIV took 0.2583034515380859 mins
Run for TEL took 0.16279316743214925 mins
Ru